In [102]:
from decimal import Decimal, getcontext
getcontext().prec = 2
class Actor:
    def __init__(self, preference, confidence):
        self.preference = preference
        self.confidence = confidence
        
def disagreement(alice, bob):
    # sum of moduli of differences
    return sum(abs(a - b) for a, b in zip(alice, bob)) / 2

def shift_factor(alice: Actor, bob: Actor):
    return alice.confidence * bob.confidence * disagreement(alice.preference, bob.preference)

def actor(pref1, confidence): 
    return Actor((pref1, 1 - pref1), confidence)

def updated_preference(actor, shift_factor):
    a1 = actor.preference[0]
    a2 = actor.preference[1]
    return a1 - shift_factor*a1/(a1+a2), actor.preference[1] - shift_factor*a2/(a1+a2), shift_factor

def from_float(f):
    return Decimal(str(f))
def build_dataset(variations):
    data = []
    for variation in variations:
        alice_pref1 = from_float(variation[0])
        alice_confidence = from_float(variation[1])
        bob_pref1 = from_float(variation[2])
        bob_confidence = from_float(variation[3])
        alice = actor(alice_pref1, alice_confidence)
        bob = actor(bob_pref1, bob_confidence)
        sf = shift_factor(alice, bob)
        
        data.append([
            variation[4],
            alice.preference, 
            bob.preference, 
            alice.confidence, 
            bob.confidence, 
            shift_factor(alice, bob),
            updated_preference(alice, sf),
            updated_preference(bob, sf)
        ])
    return data

In [103]:
import pandas as pd

variations = [
    (1.0, 1.0, 1.0, 1.0, "Agree, confident"),
    (1.0, 1.0, 0.0, 1.0, "Completely disagree, confident"),
    (0.8, 1.0, 0.0, 1.0, "Mostly disagree, confident"),
    (0.8, 0.7, 0.0, 1.0, 'Mostly disagree, Alice less confident'),
    (0.8, 0.1, 0.0, 1.0, 'Mostly disagree, Alice not confident'),
    (0.8, 0.1, 0.6, 1.0, 'Almost agree, Alice not confident, Bob confident'),
]

data = build_dataset(variations)

df = pd.DataFrame(data, columns=['Case', 'Alice', 'Bob', 'Conf', '', 'Shift', 'Updated', ''])
df

,Case,Alice,Bob,Conf,,Shift,Updated,
0,"Agree, confident","(1.0, 0.0)","(1.0, 0.0)",1.0,1.0,0.00,"(1.0, 0.00, 0.00)","(1.0, 0.00, 0.00)"
1,"Completely disagree, confident","(1.0, 0.0)","(0.0, 1.0)",1.0,1.0,1.0,"(0.0, 0.0, 1.0)","(0.0, 0.0, 1.0)"
2,"Mostly disagree, confident","(0.8, 0.2)","(0.0, 1.0)",1.0,1.0,0.80,"(0.16, 0.04, 0.80)","(0.00, 0.2, 0.80)"
3,"Mostly disagree, Alice less confident","(0.8, 0.2)","(0.0, 1.0)",0.7,1.0,0.56,"(0.35, 0.09, 0.56)","(0.00, 0.44, 0.56)"
4,"Mostly disagree, Alice not confident","(0.8, 0.2)","(0.0, 1.0)",0.1,1.0,0.080,"(0.74, 0.18, 0.080)","(0.000, 0.92, 0.080)"
5,"Almost agree, Alice not confident, Bob confident","(0.8, 0.2)","(0.6, 0.4)",0.1,1.0,0.020,"(0.78, 0.20, 0.020)","(0.59, 0.39, 0.020)"
